In [1]:
import mxnet as mx
import gluonnlp as nlp

import time
import random
import numpy as np
import sacremoses
from tqdm import tqdm_notebook as tqdm

# Local Libraries
import nmt
import dataprocessor
import utils
import nmt.gnmt_hparams

# Seeds for reproducibility
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)

# CPU setup
# ctx = mx.cpu()
# Single GPU setup
ctx = mx.gpu(0)

[nltk_data] Downloading package punkt to /home/andreto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/andreto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/andreto/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Dataset Parameters
src_lang, tgt_lang = 'en', 'de'
# No limit on sentences length
src_max_len, tgt_max_len = -1, -1

In [3]:
# WMT2016 Dataset (Train and Evaluation)
wmt_train_text_bpe = nlp.data.WMT2016BPE("train", # BPE: cheapest --> cheap@@, est
                                         src_lang=src_lang,
                                         tgt_lang=tgt_lang)

wmt_train_text     = nlp.data.WMT2016("train",
                                      src_lang=src_lang,
                                      tgt_lang=tgt_lang)

wmt_test_text_bpe  = nlp.data.WMT2016BPE("newstest2016", # BPE: cheapest --> cheap@@, est
                                         src_lang=src_lang,
                                         tgt_lang=tgt_lang)

wmt_test_text      = nlp.data.WMT2016("newstest2016",
                                     src_lang=src_lang,
                                     tgt_lang=tgt_lang)

wmt_src_vocab = wmt_train_text_bpe.src_vocab
wmt_tgt_vocab = wmt_train_text_bpe.tgt_vocab

/home/andreto/.local/lib/python3.7/site-packages/gluonnlp/vocab/vocab.py:591: UserWarning: Detected a corrupted index in the deserialize vocabulary. For versions before GluonNLP v0.7 the index is corrupted by specifying the same token for different special purposes, for example eos_token == padding_token. Deserializing the vocabulary nevertheless.
  'Detected a corrupted index in the deserialize vocabulary. '


In [4]:
# Processing datasets
# Filtering training data to a maximum number of samples,
# so that training can be handled in a reasonable time (~8 hrs)
# in single GPU setups
max_samples = int(1e6)
wmt_train_text_bpe = mx.gluon.data.SimpleDataset([wmt_train_text_bpe[i] for i in range(max_samples)])
wmt_train_text     = mx.gluon.data.SimpleDataset([wmt_train_text[i] for i in range(max_samples)])
wmt_test_text_bpe  = mx.gluon.data.SimpleDataset(wmt_test_text_bpe)
wmt_test_text      = mx.gluon.data.SimpleDataset(wmt_test_text)

In [5]:
# Dataset example (human-readable): English and German
print(wmt_test_text[16][0])
print(wmt_test_text[16][1])

By the end of the day, there would be one more death: Lamb took his own life as police closed in on him.
Bis zum Ende des Tages gab es einen weiteren Tod: Lamm nahm sich das Leben, als die Polizei ihn einkesselte.


In [6]:
# Retrieve (split) translated sequences (target)
wmt_train_tgt_sentences = wmt_train_text.transform(lambda src, tgt: tgt)
wmt_test_tgt_sentences  = wmt_test_text.transform(lambda src, tgt: tgt)
print("Sample target sentence:")
print(wmt_test_tgt_sentences[16])

Sample target sentence:
Bis zum Ende des Tages gab es einen weiteren Tod: Lamm nahm sich das Leben, als die Polizei ihn einkesselte.


In [7]:
# Dataset processing: clipping, tokenizing, indexing and adding of EOS (src/tgt) / BOS (tgt)
wmt_transform_fn = dataprocessor.TrainValDataTransform(wmt_src_vocab, wmt_tgt_vocab)

wmt_train_processed = wmt_train_text_bpe.transform(wmt_transform_fn, lazy=False)
wmt_test_processed  = wmt_test_text_bpe.transform(wmt_transform_fn, lazy=False)

wmt_train_text_with_len = wmt_train_processed.transform(nmt.utils.get_length_index_fn(), lazy=False)
wmt_test_text_with_len  = wmt_test_processed.transform(nmt.utils.get_length_index_fn(), lazy=False)

print(wmt_test_text_with_len[16][0])
print(wmt_test_text_with_len[16][1])

[ 2083 28753 16760 23875 28753 15230    28 28783 31223 12931 24017 23247
 15259   569  5971 12813 29083 20097 24348 22312 12290 24829 14439 20585
 24004 20061    62     3]
[    2  1897 31601  3259 15535  9414 18646 17382 16407 30851  9629   569
  5971 22642 23439 27119 15199  6041    28 11681 15681  7670 20454 16394
 21488 26868 28535    62     3]


In [8]:
# Batcher
wmt_batchify_fn = nlp.data.batchify.Tuple(
    nlp.data.batchify.Pad(),                   # Source Token IDs
    nlp.data.batchify.Pad(),                   # Target Token IDs
    nlp.data.batchify.Stack(dtype='float32'),  # Source Sequence Length
    nlp.data.batchify.Stack(dtype='float32'),  # Target Sequence Length
    nlp.data.batchify.Stack())                 # Index

/home/andreto/.local/lib/python3.7/site-packages/gluonnlp/data/batchify/batchify.py:235: UserWarning: Padding value is not given and will be set automatically to 0 in data.batchify.Pad(). Please check whether this is intended (e.g. value of padding index in the vocabulary).
  'Padding value is not given and will be set automatically to 0 '


In [9]:
# Hyperparameters
hparams = nmt.gnmt_hparams

In [10]:
# Samplers
wmt_train_batch_sampler = nlp.data.FixedBucketSampler(
    lengths=wmt_train_text_with_len.transform(lambda src, tgt, src_len, tgt_len, idx: (src_len, tgt_len)),
    num_buckets=hparams.num_buckets,
    batch_size=hparams.batch_size)
print(wmt_train_batch_sampler.stats())

wmt_test_batch_sampler = nlp.data.FixedBucketSampler(
    lengths=wmt_test_text_with_len.transform(lambda src, tgt, src_len, tgt_len, idx: (src_len, tgt_len)),
    num_buckets=hparams.num_buckets,
    batch_size=hparams.test_batch_size)
print(wmt_test_batch_sampler.stats())

FixedBucketSampler:
  sample_num=1000000, batch_num=15628
  key=[(28, 29), (55, 56), (82, 83), (109, 110), (136, 137)]
  cnt=[484394, 419332, 89416, 6836, 22]
  batch_size=[64, 64, 64, 64, 64]
FixedBucketSampler:
  sample_num=2999, batch_num=97
  key=[(23, 26), (43, 48), (63, 70), (83, 92), (103, 114)]
  cnt=[1417, 1191, 329, 56, 6]
  batch_size=[32, 32, 32, 32, 32]


In [11]:
# DataLoaders
wmt_train_data_loader = mx.gluon.data.DataLoader(
    wmt_train_text_with_len,
    batch_sampler=wmt_train_batch_sampler,
    batchify_fn=wmt_batchify_fn,
    num_workers=8)
print('Number of training batches:', len(wmt_train_data_loader))

wmt_test_data_loader = mx.gluon.data.DataLoader(
    wmt_test_text_with_len,
    batch_sampler=wmt_test_batch_sampler,
    batchify_fn=wmt_batchify_fn,
    num_workers=8)
print('Number of testing batches:', len(wmt_test_data_loader))

Number of training batches: 15628
Number of testing batches: 97


In [12]:
# Model
encoder, decoder, one_step_ahead_decoder = nmt.gnmt.get_gnmt_encoder_decoder(
    hidden_size=hparams.num_hidden,
    dropout=hparams.dropout,
    num_layers=hparams.num_layers,
    num_bi_layers=hparams.num_bi_layers)

gnmt_model = nlp.model.translation.NMTModel(
    src_vocab=wmt_src_vocab,
    tgt_vocab=wmt_tgt_vocab,
    encoder=encoder,
    decoder=decoder,
    one_step_ahead_decoder=one_step_ahead_decoder,
    embed_size=hparams.num_hidden,
    prefix='gnmt_')

gnmt_model.initialize(init=mx.init.Uniform(0.1), ctx=ctx)
static_alloc = True
gnmt_model.hybridize(static_alloc=static_alloc)

In [13]:
scorer=nlp.model.BeamSearchScorer(
    alpha=hparams.lp_alpha,
    K=hparams.lp_k)

gnmt_translator = nmt.translation.BeamSearchTranslator(
    model=gnmt_model,
    beam_size=hparams.beam_size,
    scorer=scorer,
    max_length=tgt_max_len + 100)

print("Use beam_size={}, alpha={}, K={}".format(hparams.beam_size, hparams.lp_alpha, hparams.lp_k))

Use beam_size=10, alpha=1.0, K=5


In [14]:
# Evaluation (Baseline)
eval_start_time = time.time()
wmt_loss_function = nlp.loss.MaskedSoftmaxCELoss()
wmt_loss_function.hybridize()
wmt_detokenizer = nlp.data.SacreMosesDetokenizer()

gnmt_test_loss, gnmt_test_translation_out = nmt.utils.evaluate(
    gnmt_model,
    wmt_test_data_loader,
    wmt_loss_function,
    gnmt_translator,
    wmt_tgt_vocab,
    wmt_detokenizer,
    ctx)

gnmt_test_bleu_score, _, _, _, _ = nmt.bleu.compute_bleu(
    [wmt_test_tgt_sentences],
    gnmt_test_translation_out,
    tokenized=False,
    tokenizer=hparams.bleu,
    split_compound_word=False,
    bpe=False)

print('WMT16 EN-DE SOTA model test loss: %.2f; test bleu score: %.2f; time cost %.2fs' %(gnmt_test_loss, gnmt_test_bleu_score * 100, (time.time() - eval_start_time)))

  0%|          | 0/97 [00:00<?, ?it/s]

Extension horovod.torch has not been built: /home/ubuntu/anaconda3/envs/mxnet_p37/lib/python3.7/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-37m-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2022-06-04 21:12:09.664 ip-172-31-28-47:2314 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-06-04 21:12:09.694 ip-172-31-28-47:2314 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
WMT16 EN-DE SOTA model test loss: 7.75; test bleu score: 0.00; time cost 125.44s


In [15]:
# Training
trainer = mx.gluon.Trainer(gnmt_model.collect_params(), 'adam', {'learning_rate': hparams.lr})

In [16]:
nmt.utils.train(
    gnmt_model,
    wmt_train_data_loader,
    wmt_test_data_loader,
    wmt_loss_function,
    trainer,
    gnmt_translator,
    wmt_tgt_vocab,
    wmt_detokenizer,
    wmt_test_tgt_sentences,
    hparams.save_dir,
    hparams,
    ctx)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/15628 [00:00<?, ?it/s]

[Epoch 0 Batch 10/15628] loss=9.6243, ppl=15128.5737, gnorm=1.9159, throughput=12.34K wps, wc=99.61K
[Epoch 0 Batch 20/15628] loss=7.9087, ppl=2720.7845, gnorm=0.6544, throughput=19.67K wps, wc=105.05K
[Epoch 0 Batch 30/15628] loss=7.6519, ppl=2104.5417, gnorm=0.4327, throughput=19.34K wps, wc=105.35K
[Epoch 0 Batch 40/15628] loss=7.5384, ppl=1878.8451, gnorm=0.2846, throughput=20.33K wps, wc=105.56K
[Epoch 0 Batch 50/15628] loss=7.5069, ppl=1820.6161, gnorm=0.2547, throughput=20.39K wps, wc=105.68K
[Epoch 0 Batch 60/15628] loss=7.4718, ppl=1757.8342, gnorm=0.2675, throughput=20.41K wps, wc=105.29K
[Epoch 0 Batch 70/15628] loss=7.4867, ppl=1784.1696, gnorm=0.2836, throughput=20.22K wps, wc=105.16K
[Epoch 0 Batch 80/15628] loss=7.4260, ppl=1679.0376, gnorm=0.2879, throughput=20.06K wps, wc=105.24K
[Epoch 0 Batch 90/15628] loss=7.3874, ppl=1615.4975, gnorm=0.3212, throughput=19.21K wps, wc=104.38K
[Epoch 0 Batch 100/15628] loss=7.3273, ppl=1521.2603, gnorm=0.2824, throughput=19.18K wps, 

[Epoch 0 Batch 830/15628] loss=5.2158, ppl=184.1577, gnorm=0.2711, throughput=19.23K wps, wc=77.95K
[Epoch 0 Batch 840/15628] loss=5.2508, ppl=190.7132, gnorm=0.2761, throughput=18.61K wps, wc=78.97K
[Epoch 0 Batch 850/15628] loss=5.1554, ppl=173.3676, gnorm=0.2798, throughput=18.69K wps, wc=77.99K
[Epoch 0 Batch 860/15628] loss=5.1844, ppl=178.4695, gnorm=0.2858, throughput=20.21K wps, wc=79.14K
[Epoch 0 Batch 870/15628] loss=5.1232, ppl=167.8785, gnorm=0.2685, throughput=18.62K wps, wc=78.52K
[Epoch 0 Batch 880/15628] loss=5.0713, ppl=159.3846, gnorm=0.2799, throughput=19.24K wps, wc=77.92K
[Epoch 0 Batch 890/15628] loss=5.2012, ppl=181.4908, gnorm=0.2749, throughput=18.37K wps, wc=78.71K
[Epoch 0 Batch 900/15628] loss=5.1875, ppl=179.0148, gnorm=0.2652, throughput=19.21K wps, wc=79.19K
[Epoch 0 Batch 910/15628] loss=5.1658, ppl=175.1717, gnorm=0.2670, throughput=18.73K wps, wc=79.05K
[Epoch 0 Batch 920/15628] loss=5.1539, ppl=173.0987, gnorm=0.2533, throughput=18.54K wps, wc=78.10K


[Epoch 0 Batch 1650/15628] loss=4.6844, ppl=108.2469, gnorm=0.3247, throughput=17.10K wps, wc=47.66K
[Epoch 0 Batch 1660/15628] loss=4.7090, ppl=110.9453, gnorm=0.3314, throughput=17.29K wps, wc=47.13K
[Epoch 0 Batch 1670/15628] loss=4.7722, ppl=118.1776, gnorm=0.3263, throughput=17.38K wps, wc=46.58K
[Epoch 0 Batch 1680/15628] loss=4.5803, ppl=97.5462, gnorm=0.3160, throughput=17.74K wps, wc=47.93K
[Epoch 0 Batch 1690/15628] loss=4.6665, ppl=106.3281, gnorm=0.3271, throughput=17.72K wps, wc=47.36K
[Epoch 0 Batch 1700/15628] loss=4.6613, ppl=105.7778, gnorm=0.3208, throughput=18.04K wps, wc=47.34K
[Epoch 0 Batch 1710/15628] loss=4.6217, ppl=101.6682, gnorm=0.3146, throughput=18.66K wps, wc=47.34K
[Epoch 0 Batch 1720/15628] loss=4.6342, ppl=102.9432, gnorm=0.3101, throughput=17.01K wps, wc=46.90K
[Epoch 0 Batch 1730/15628] loss=4.5949, ppl=98.9793, gnorm=0.3097, throughput=15.84K wps, wc=46.05K
[Epoch 0 Batch 1740/15628] loss=4.7189, ppl=112.0496, gnorm=0.3175, throughput=17.81K wps, wc

...

[Epoch 0 Batch 13950/15628] loss=2.0337, ppl=7.6425, gnorm=0.3892, throughput=16.36K wps, wc=22.10K
[Epoch 0 Batch 13960/15628] loss=2.1768, ppl=8.8182, gnorm=0.4136, throughput=16.68K wps, wc=22.62K
[Epoch 0 Batch 13970/15628] loss=2.0621, ppl=7.8623, gnorm=0.3934, throughput=16.03K wps, wc=22.39K
[Epoch 0 Batch 13980/15628] loss=2.0446, ppl=7.7259, gnorm=0.3874, throughput=16.23K wps, wc=22.24K
[Epoch 0 Batch 13990/15628] loss=1.8817, ppl=6.5643, gnorm=0.3850, throughput=16.13K wps, wc=22.21K
[Epoch 0 Batch 14000/15628] loss=2.0484, ppl=7.7555, gnorm=0.3912, throughput=16.16K wps, wc=22.35K
[Epoch 0 Batch 14010/15628] loss=1.9250, ppl=6.8548, gnorm=0.3922, throughput=15.07K wps, wc=21.73K
[Epoch 0 Batch 14020/15628] loss=2.0074, ppl=7.4442, gnorm=0.3906, throughput=16.68K wps, wc=22.40K
[Epoch 0 Batch 14030/15628] loss=1.9992, ppl=7.3833, gnorm=0.4023, throughput=16.37K wps, wc=22.21K
[Epoch 0 Batch 14040/15628] loss=1.9767, ppl=7.2188, gnorm=0.3881, throughput=16.55K wps, wc=22.22K


[Epoch 0 Batch 14770/15628] loss=2.0156, ppl=7.5052, gnorm=0.3913, throughput=16.38K wps, wc=22.48K
[Epoch 0 Batch 14780/15628] loss=2.0455, ppl=7.7329, gnorm=0.3856, throughput=16.27K wps, wc=22.69K
[Epoch 0 Batch 14790/15628] loss=2.0331, ppl=7.6379, gnorm=0.3819, throughput=17.42K wps, wc=23.27K
[Epoch 0 Batch 14800/15628] loss=2.0831, ppl=8.0290, gnorm=0.3931, throughput=14.97K wps, wc=21.92K
[Epoch 0 Batch 14810/15628] loss=1.9286, ppl=6.8797, gnorm=0.3919, throughput=15.68K wps, wc=21.67K
[Epoch 0 Batch 14820/15628] loss=2.1469, ppl=8.5581, gnorm=0.3915, throughput=15.97K wps, wc=23.00K
[Epoch 0 Batch 14830/15628] loss=2.0580, ppl=7.8303, gnorm=0.3957, throughput=15.71K wps, wc=21.80K
[Epoch 0 Batch 14840/15628] loss=2.1603, ppl=8.6739, gnorm=0.3738, throughput=17.00K wps, wc=23.45K
[Epoch 0 Batch 14850/15628] loss=2.0037, ppl=7.4166, gnorm=0.3942, throughput=15.68K wps, wc=22.01K
[Epoch 0 Batch 14860/15628] loss=1.8585, ppl=6.4142, gnorm=0.3792, throughput=15.95K wps, wc=21.69K


[Epoch 0 Batch 15590/15628] loss=1.9158, ppl=6.7923, gnorm=0.3767, throughput=14.94K wps, wc=22.43K
[Epoch 0 Batch 15600/15628] loss=1.9353, ppl=6.9259, gnorm=0.3769, throughput=15.14K wps, wc=22.72K
[Epoch 0 Batch 15610/15628] loss=2.0613, ppl=7.8564, gnorm=0.3840, throughput=15.04K wps, wc=22.36K
[Epoch 0 Batch 15620/15628] loss=2.0846, ppl=8.0413, gnorm=0.3871, throughput=16.09K wps, wc=22.37K


  0%|          | 0/97 [00:00<?, ?it/s]

NameError: name 'val_tgt_sentences' is not defined

In [ ]:
from importlib import reload

In [ ]:
reload(nmt)
reload(nmt.utils)
reload(nmt.gnmt_hparams)